In [3]:
%load_ext autoreload
%autoreload 2

In [106]:
from dataclasses import dataclass

import numpy as np
from scipy.stats import bernoulli

In [72]:
def make_binomial_generator(p):
    def binomial_generator(n_periods):
        return 2*(bernoulli.rvs(p, size=n_periods)-0.5)
    return binomial_generator

def make_constant_fraction_strategy(fraction):
    def strategy(wealth):
        return fraction*wealth
    return strategy

def sim_wealth(outcome_generator, strategy, *, n_periods=1):
    outcomes = outcome_generator(n_periods)
    wealth = np.zeros(n_periods + 1)
    wealth[0] = 1
    for i in range(1, n_periods + 1):
        wealth[i] = wealth[i-1] + strategy(wealth[i-1]) * outcomes[i-1]
        if wealth[i] <= 0:
            wealth[i] = 0
            break
    return wealth


In [85]:
sim_wealth(make_binomial_generator(0.8), make_constant_fraction_strategy(1), n_periods=100)

array([1.000000e+00, 2.000000e+00, 4.000000e+00, 8.000000e+00,
       1.600000e+01, 3.200000e+01, 6.400000e+01, 1.280000e+02,
       2.560000e+02, 5.120000e+02, 1.024000e+03, 2.048000e+03,
       4.096000e+03, 8.192000e+03, 1.638400e+04, 3.276800e+04,
       6.553600e+04, 1.310720e+05, 2.621440e+05, 5.242880e+05,
       1.048576e+06, 2.097152e+06, 4.194304e+06, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.0000

In [130]:
binomial_60_generator = make_binomial_generator(0.6)
all_in = make_constant_fraction_strategy(0.5)
n_sims = 10000
n_periods = 10

@dataclass
class Cumulator:
    n_bankrupt = 0
    cum_wealth = 0

cumul = Cumulator()
for i in range(n_sims):
    wealth = sim_wealth(binomial_60_generator, all_in, n_periods=n_periods)
    cumul.cum_wealth += wealth[n_periods]
    if wealth[n_periods] < 0.0001:
        cumul.n_bankrupt += 1

{
    "prob_brankrupt": cumul.n_bankrupt / n_sims,
    "avg_wealth": cumul.cum_wealth / n_sims,
}

{'prob_brankrupt': 0.0, 'avg_wealth': 2.5781560546875}